In [ ]:
import os
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Input

# ------------------------------
# 1️⃣ **Mount Google Drive**
# ------------------------------
from google.colab import drive
drive.mount('/content/drive')

# ------------------------------
# 2️⃣ **Load .npy Files into Memory**
# ------------------------------
DATA_DIR = "/content/drive/MyDrive/real_cloud_data"

def load_data(data_dir):
    """Load images and labels from .npy files."""
    X, y = [], []

    for label in sorted(os.listdir(data_dir)):  # Folders named 1, 2, 3, ...
        class_dir = os.path.join(data_dir, label)

        if os.path.isdir(class_dir):
            for npy_file in os.listdir(class_dir):
                if npy_file.endswith('.npy'):
                    file_path = os.path.join(class_dir, npy_file)
                    data = np.load(file_path)

                    # Ensure valid image format
                    if data.ndim == 3:  # (H, W, C)
                        X.append(data)
                        y.append(label)

    return np.array(X), np.array(y)

# Load the dataset
X, y = load_data(DATA_DIR)
print(f"✅ Loaded {len(X)} samples with {len(np.unique(y))} classes")

# ------------------------------
# 3️⃣ **Preprocess the Data**
# ------------------------------
# Normalize the images for FaceNet
X = X / 255.0

# Encode class labels into integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

# ------------------------------
# 4️⃣ **Load Pretrained FaceNet Model**
# ------------------------------
def build_facenet_model():
    base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

facenet = build_facenet_model()
print("✅ FaceNet model loaded successfully!")

# ------------------------------
# 5️⃣ **Resize and Fix Image Channels**
# ------------------------------
def resize_images(images, target_size=(160, 160)):
    """Resize images to 160x160 and ensure 3 channels."""
    resized_images = []

    for img in images:
        img_resized = tf.image.resize(img, target_size)

        # Ensure it has 3 channels (convert RGBA to RGB)
        if img_resized.shape[-1] == 4:
            img_resized = img_resized[..., :3]  # Keep only RGB channels

        resized_images.append(img_resized)

    return np.array(resized_images)

# Resize images
X_train_resized = resize_images(X_train)
X_test_resized = resize_images(X_test)

# ------------------------------
# 6️⃣ **Extract Face Embeddings**
# ------------------------------
def get_embeddings(model, images):
    """Get face embeddings and handle 4-channel images."""
    # Ensure all images have 3 channels
    images_fixed = np.array([img[..., :3] if img.shape[-1] == 4 else img for img in images])

    embeddings = model.predict(images_fixed)
    return embeddings

# Extract face embeddings
train_embeddings = get_embeddings(facenet, X_train_resized)
test_embeddings = get_embeddings(facenet, X_test_resized)

# ------------------------------
# 7️⃣ **Train the Classifier**
# ------------------------------
# Normalize embeddings
scaler = StandardScaler()
train_embeddings = scaler.fit_transform(train_embeddings)
test_embeddings = scaler.transform(test_embeddings)

# Use SVM for classification
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(train_embeddings, y_train)

# ------------------------------
# 8️⃣ **Evaluate the Model**
# ------------------------------
y_pred = svm_classifier.predict(test_embeddings)

# Print accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print("\n✅ Classification Accuracy:", accuracy)
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# ------------------------------
# 9️⃣ **Create Folder & Save Models**
# ------------------------------
# Define directory to save models
SAVE_DIR = "/content/drive/MyDrive/saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# Save models and preprocessing files into the directory
joblib.dump(svm_classifier, os.path.join(SAVE_DIR, 'svm_classifier.pkl'))
joblib.dump(scaler, os.path.join(SAVE_DIR, 'scaler.pkl'))
joblib.dump(le, os.path.join(SAVE_DIR, 'label_encoder.pkl'))
facenet.save(os.path.join(SAVE_DIR, 'facenet_model.h5'))

print("\n✅ All models and encoders are saved in:", SAVE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded 240 samples with 12 classes
Train size: 192, Test size: 48
✅ FaceNet model loaded successfully!
6/6 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 9s/step



✅ Classification Accuracy: 0.9166666666666666

📊 Classification Report:
               precision    recall  f1-score   support

           1       0.88      1.00      0.93         7
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         3
           2       1.00      0.67      0.80         3
           3       0.80      0.80      0.80         5
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         3
           8       0.50      1.00      0.67         2
           9       1.00      0.71      0.83         7

    accuracy                           0.92        48
   macro avg       0.93      0.93      0.92        48
weighted avg       0.94      0.92      0.92        48


✅ All models and encoders are saved in: /content/drive/MyD

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive
